# PDH Signal in Finesse

In this notebook, you will learn how modulation and demodulation can be used in Finesse to produce Pound-Drever-Hall error signals and (if there is time) simple active feedback loops within a model.

## Goal of the session
In this notebook, we return to our model of a 2-mirror cavity. We'll first introduce sidebands to the input field, then use these to generate a Pound-Drever-Hall error signal that could be used to lock the laser to the cavity (or vice versa). 

*If there is time*, we'll then take a look at the `lock` command included in Finesse, which can be used to create a simple feedback loop in your simulations.

**Finesse/Pykat things you will learn**:

 - constructing an error signal using the `mod` and `pd1` components
 - manipulating an existing `kat` object using PyKat commands like `kat.[component].remove()`


## Getting Started

As usual, we start by loading PyKat and the other Python packages we will need. This time, we *also* load the code for a simple 2-mirror cavity, as used in our earlier notebook:

<img src=http://www.gwoptics.org/learn/02_Plane_waves/01_Fabry_Perot_cavity/fabry-perot.svg width=40%>

<font size=1 color='Grey'>image: GWOptics</font>

For the tasks below, you should create a copy of this optical model using `kat = basekat.deepcopy`.

(type shift+enter to run)

In [1]:
# from pykat import finesse        # import the whole pykat.finesse package
# from pykat.commands import *     # import all packages in pykat.commands
# import numpy as np               # for basic math/sci/array functions
# import matplotlib.pyplot as plt  # for plotting

# # tell the notebook to automatically show plots inline below each cell
# %matplotlib inline               
# # use pykat's plotting style. change dpi to change plot sizes on your screen
# pykat.init_pykat_plotting(dpi=90)

# #initialise Finesse with a new empty kat object
# basekat = finesse.kat() 
# #tell Finesse to talk less
# basekat.verbose = False
# #parse the Finesse code into PyKat
# basekat.parse("""
# l laser 1 0 n0
# s s0 0.1 n0 n1

# #the cavity:
# m1 M1 0.15 0 0 n1 n2
# s scav 0.2 n2 n3
# m1 M2 0.15 0 0 n3 n4
# """)

# print(basekat)

## Error signals, modulation and demodulation

In a Finesse simulation, a cavity will sit perfectly on resonance unless we deliberately alter some parameter that changes the resonant condition of the system. In reality, external disturbances such as seismic noise, temperature changes, or drifts in the input laser frequency will affect the system. The optical system must be actively controlled so that the cavity length and laser frequency are *locked* such that one follows the other, and the cavity stays resonant. 

To do this, we require an error signal that can be used in a feedback loop. 

As you have seen, one method to do this is to modulate the input field, producing <a href=https://en.wikipedia.org/wiki/Sideband>sidebands</a> which interact with the cavity differently to the main 'carrier' field. The beat signal produced between these sidebands and the carrier, after interacting with the cavity, depends on the resonant condition in the cavity for all of the fields involved.

As you have seen, the 'standard' method for locking a laser to a two mirror cavity is the <a href=https://en.wikipedia.org/wiki/Pound–Drever–Hall_technique>Pound-Drever-Hall (PDH)</a> technique, depicted below. In this case, the input beam is phase modulated to produce a pair of sidebands which do not resonate in the cavity, so are just directly reflected. A photodiode on reflection then measures the beat signal, which is demodulated and low-passed to produce an antisymmetric DC error signal that is linear over the bandwidth of the cavity. This error signal can be fed back to a component that can change the phase of the light, e.g. the laser.

<img src=https://upload.wikimedia.org/wikipedia/commons/thumb/6/6c/Pound–Drever–Hall_technique.svg/400px-Pound–Drever–Hall_technique.svg.png width='40%' height='40%'>
<img src=https://www.newport.com/medias/sys_master/images/images/h59/h68/8797626368030/App-note-15-Fig3-600w.gif width=30% height=30%>
<font size=1 color='Grey'>images: Wikipedia, Newport Optics</font>

You can learn more about modulation and demodulation in <a href=https://link.springer.com/article/10.1007%2Fs41114-016-0002-8#Sec16>chapter 3</a> of Interferometer Techniques, or by exploring <a href=http://www.gwoptics.org/learn/02_Plane_waves/03_Optical_modulation/index.html> Sections 2.3 and 2.4</a> of the GWOptics "Learn Interferometry" course.

**In Finesse** the full PDH technique is replicated using two components: (1) a modulator (`mod`), which can be configured to act like a EOM, EOAM, produce a single sideband, etc; and (2) a demodulating photodiode (`pd[n]`), where `n` specifies the number of demodulations required (1, in our case). The `pd1` combines the roles of the photodiode, mixer, and low-pass filter in a physical setup, directly recording the resulting error signal. 


<div style="background-color:#cceffc;padding:10px;">

### Task 1: Plot the PDH error signal for the cavity

1. Create a deep copy of the `basekat` object to load the simple cavity model.


2. Remove the existing space between the laser and cavity, `s0`, by using the pykat command `kat.s0.remove()`. 
    (You can check this has worked using `print(kat)` to see the current kat object).


3. In the gap that now exists between the laser and the cavity, parse the Finesse commands for a space, followed by a *phase* modulator with modulation depth 0.3 and frequency 80MHz, followed by a second space. Ensure that the nodal network is fully reconnected after this replacement.


4. Parse the code for a photodiode with one demodulation at the modulator frequency, looking on reflection from the cavity.


5. Plot the photodiode output as a function of the tuning of mirror M2 in the range -50 to +50 degrees.


6. How does the error signal change with demodulation phase? What is the optimal phase in this case? What other parameters affect the size and shape of the error signal? Why?

**Hint:** you might like to check the <a href=http://www.gwoptics.org/finesse/reference/>Finesse syntax reference</a>.

**Hint:** in Finesse, large numbers can be input using the metric prefixes, so e.g. an EOM frequency of 1kHz can be specified as `1k`. 
 </div>

In [2]:
from pykat import finesse        # import the whole pykat.finesse package
from pykat.commands import *     # import all packages in pykat.commands
import numpy as np               # for basic math/sci/array functions
import matplotlib.pyplot as plt  # for plotting

# tell the notebook to automatically show plots inline below each cell
%matplotlib inline               
# use pykat's plotting style. change dpi to change plot sizes on your screen
pykat.init_pykat_plotting(dpi=90)

basekat = finesse.kat()    # Initialising FINESSE
basekat.verbose = False    # Keep quiet

kat = deepcopy(basekat)

#parse the Finesse code into PyKat
kat.parse("""
l laser 1 0 nin                          % Laser P=1W f_offset=0Hz
mod EOM 15M 0.001 1 pm nin n0            % Phase modulator f_mod=15MHz, modulation index=0.001
maxtem 3                                 % Include HG modes up to order 3
s s0 0 n0 n1
bs pickoff 0.1 0.9 0 45 n1 dump n2 n3    % Beam splitter with R=0.1, T=0.9 at zero tuning
s s1 0.1 n2 n4                           % Space 10cm length
m ITM 0.99 0.01 0 n4 n5                  % Cavity input mirror with R=0.99, T=0.01, at zero tuning
s scavity 1 n5 n6                        % Space 1m length
m ETM 0.99 0.01 0 n6 n7                  % Cavity end mirror with R=0.99, T=0.01, at zero tuning
  
s s2 0.1 n3 n8                           % Space 10cm length
bs bsQPD 0.5 0.5 0 45 n8 n9 n10 dump     % Beam splitter with R=T=0.5 at zero tuning
s sQPD1 0.1 n9 n11                       % Space 10cm length
s sQPD2 0.1 n10 n12                      % Space 10cm length
  
cav cavity ITM n5 ETM n6                 % Trace cavity eigenmode for the cavity (mirrors ITM and ETM)
trace 2                                  % Display results of cavity trace to terminal output
attr ETM Rc 1.3                          % Add curvature with radius 1.3m to mirror ETM
attr ETM xbeta 0                         % Add a variable for the angular tuning of mirror ETM
attr ITM xbeta 0                         % Add a variable for the angular tuning of mirror ITM
pd1 PDrefl 15M 0 n8*                     % Photodetector with demodulation at 15MHz, demodulation phase 0
  
attr sQPD1 g 40                          % Fix the Gouy phase shift the space QPDsplit2QPDrefl1 to 40 degrees
attr sQPD2 g 130                         % Add 90 deg Gouy phase shift to the space QPDsplit2QPDrefl2
  
pd1 QPDrefl1 15M 0 n11                   % Photodetector with demodulation at 15MHz, demod. phase 0 deg
%pdtype QPDrefl1 x-split                  % Specify that QPDrefl1 is a split photodiode
  
pd1 QPDrefl2 15M 0 n12                   % Photodetector with demodulation at 15MHz, demod. phase 0 deg
%pdtype QPDrefl2 x-split                  % Specify that QPDrefl2 is a split photodiode
  
set err PDrefl re                        % Define the signal from PDrefl to be used to generate error signal
lock z $err 3000 10n                     % Generate feedback signal, gain of 3000, lock accuracy of 10ppm
put ETM phi $z                           % Apply feedback signal to the tuning of mirror ETM
noplot z                                 % Don't plot the feedback signal
%xaxis ITM xbeta lin 0 0.8m 500          % Sweep misalignment of mirror ITM from 0 to 0.8mrad
xaxis ETM xbeta lin 0 0.8m 500           % Sweep misalignment of mirror ETM from 0 to 0.8mrad

""")

print(kat)

out = kat.run()



    


                                              ..-
    PyKat 1.1.331         _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat

% Generated by PyKat 24.08.2020 14:18:29

l laser 1.0 0.0 0.0 nin
mod EOM 15000000.0 0.001 1 pm 0.0 nin n0
s s0 0.0 n0 n1
bs pickoff 0.1 0.9 0.0 45.0 n1 dump n2 n3
s s1 0.1 n2 n4
m ITM 0.99 0.01 0.0 n4 n5
attr ITM xbeta 0.0
s scavity 1.0 n5 n6
m ETM 0.99 0.01 0.0 n6 n7
attr ETM Rcx 1.3
attr ETM Rcy 1.3
attr ETM xbeta 0.0
s s2 0.1 n3 n8
bs bsQPD 0.5 0.5 0.0 45.0 n8 n9 n10 dump
s sQPD1 0.1 n9 n11
attr sQPD1 gx 40
attr sQPD1 gy 40
s sQPD2 0.1 n10 n12
attr sQPD2 gx 130
attr sQPD2 gy 130
cav cavity ITM n5 ETM n6
pd1 PDrefl 15000000.0 0.0 n8*
pd1 QPDrefl1 15000000.0 0.0 n11
pd1 QPDrefl2 15000000.